In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import os
import requests
import matplotlib.pyplot as plt
import pyproj
import datetime
import json
%matplotlib inline

In [14]:
acs_results_2016 = requests.get('https://api.census.gov/data/2016/acs/acs1/profile?get=NAME,DP05_0001E,DP02_0001PE,DP05_0009PE,DP05_0066PE,DP05_0032PE,DP05_0033PE,DP05_0017E,DP04_0134E,DP03_0062E,DP02_0064PE&for=place:*&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4')

In [3]:
acs_results_2016.status_code

200

In [15]:
data = acs_results_2016.text
total_df = pd.read_json(data)

In [16]:
total_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,NAME,DP05_0001E,DP02_0001PE,DP05_0009PE,DP05_0066PE,DP05_0032PE,DP05_0033PE,DP05_0017E,DP04_0134E,DP03_0062E,DP02_0064PE,state,place
1,"Birmingham city, Alabama",213434,90117,17.2,3.6,24.6,71.3,36.0,777,36241,16.9,01,07000
2,"Dothan city, Alabama",67714,26119,13.6,2.9,60.9,35.0,38.9,720,42336,14.7,01,21184
3,"Hoover city, Alabama",84943,32110,15.3,7.6,70.9,16.7,36.6,1021,79004,34.1,01,35896
4,"Huntsville city, Alabama",196225,80540,14.3,6.3,60.6,31.5,35.6,766,51895,25.7,01,37000


In [17]:
total_df.columns = total_df.iloc[0]
total_df.drop([0], inplace = True) 
variables = {'DP05_0001E':"population",'DP02_0001PE':"total_hh",'DP05_0009PE':"pct_25_34",'DP05_0003PE':"pct_women",'DP05_0066PE': "pct_hispanic",
             'DP05_0032PE': "pct_white",'DP05_0033PE':"pct_black",'DP05_0017E':"median_age",'DP04_0134E':"gross_rent",
             'DP03_0062E': "median_hh_income",'DP02_0064PE':"pct_bachelor"}
total_df.rename(columns = variables, inplace=True)
total_df['year'] = 2016
total_df['avg_hh_size'] = total_df['population']/total_df['total_hh']

In [18]:
total_df.head()

,NAME,population,total_hh,pct_25_34,pct_hispanic,pct_white,pct_black,median_age,gross_rent,median_hh_income,pct_bachelor,state,place,year
1,"Birmingham city, Alabama",213434,90117,17.2,3.6,24.6,71.3,36.0,777,36241,16.9,01,07000,2016
2,"Dothan city, Alabama",67714,26119,13.6,2.9,60.9,35.0,38.9,720,42336,14.7,01,21184,2016
3,"Hoover city, Alabama",84943,32110,15.3,7.6,70.9,16.7,36.6,1021,79004,34.1,01,35896,2016
4,"Huntsville city, Alabama",196225,80540,14.3,6.3,60.6,31.5,35.6,766,51895,25.7,01,37000,2016
5,"Mobile city, Alabama",192895,79188,15.8,2.5,46.4,49.8,36.9,804,39463,18.9,01,50000,2016
